In [ ]:
import importlib
import business
importlib.reload(business)

In [ ]:
# 第一轮规则审核代码 
import pandas as pd
import os
from datetime import datetime
import re
from typing import Tuple, Dict, List
from business import validate_business
from tqdm import tqdm

def first_round_rule_audit(input_file: str) -> pd.DataFrame:
    """
    执行第一轮规则审核并返回完整的审核结果DataFrame
    
    Args:
        input_file: 输入Excel文件路径
    
    Returns:
        df: 包含所有短信审核结果的DataFrame
    """
    # 读取Excel文件
    print(f"读取文件: {input_file}")
    df = pd.read_excel(input_file)
    print(f"成功读取文件，共 {len(df)} 条记录")
    
    # 创建结果列
    df['总体操作类型'] = None
    df['业务操作类型'] = None
    
    # 审核计数器
    pass_count = 0
    fail_count = 0
    error_count = 0
    
    print("=============== 开始执行第一轮：规则审核==============")
    
    # 对每条短信执行规则审核
    for index, row in tqdm(df.iterrows(), total=len(df), desc="规则审核进度", ncols=100):
        
            
        try:
            # 调用业务规则审核
            business_passed, business_reason = validate_business(
                row['产品类型'],
                row['短信内容'], 
                row['短信签名'],
                row.get('账户类型')
            )
            
            # 更新结果
            if business_passed:
                df.loc[index, '总体操作类型'] = '放行'
                pass_count += 1
            else:
                df.loc[index, '总体操作类型'] = '失败'
                fail_count += 1
                
            df.loc[index, '业务操作类型'] = business_reason
            
        except Exception as e:
            print(f"处理行 {index} 时出错: {str(e)}")
            df.loc[index, '总体操作类型'] = '处理错误'
            df.loc[index, '业务操作类型'] = f"处理错误: {str(e)}"
            error_count += 1
    
    # 输出统计信息
    total = len(df)
    print(f"\n规则审核完成:")
    print(f"- 总记录数: {total}")
    print(f"- 放行数量: {pass_count} ({pass_count/total*100:.2f}%)")
    print(f"- 失败数量: {fail_count} ({fail_count/total*100:.2f}%)")
    if error_count > 0:
        print(f"- 错误数量: {error_count} ({error_count/total*100:.2f}%)")
    
    #可选：保存结果到Excel文件
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"123规则审核_{timestamp}.xlsx"
    df.to_excel(output_file, index=False)
    print(f"完整审核结果已保存至: {output_file}")
    
    
    # 统计不同情况的数量
    code_pass_human_fail = len(df[(df['总体操作类型'] == '放行') & (df['操作类型'] == '失败')])
    code_fail_human_pass = len(df[(df['总体操作类型'] == '失败') & (df['操作类型'] == '放行')])
    matched = len(df[df['总体操作类型'] == df['操作类型']])
    match_rate = (matched / total) * 100
    
    # 打印结果
    print('\n审核结果统计:')
    print(f'总样本数: {total}')
    print(f'代码放行但人工失败数量: {code_pass_human_fail}')
    print(f'代码失败但人工放行数量: {code_fail_human_pass}')
    print(f'匹配数量: {matched}')
    print(f'匹配率: {match_rate:.2f}%')


    return df

# 使用示例 
df_with_results = first_round_rule_audit("3月审核记录.xlsx")



In [ ]:
import importlib
import business
import ai_check 
importlib.reload(business)
importlib.reload(ai_check )

In [156]:
import psutil
import os

def show_memory_usage():
    process = psutil.Process(os.getpid())
    print(f"当前内存使用: {process.memory_info().rss / 1024 / 1024:.2f} MB")

In [ ]:

import time
from business import validate_business
from typing import Tuple, Dict
import pandas as pd
import os
from datetime import datetime
import re
from collections import Counter
from ai_check import AIAuditor
import json

# 2. 准备AI审核数据
def prepare_ai_data(sample_df):
    """准备AI审核数据"""
    ai_audit_list = []
    for _, row in sample_df.iterrows():
        try:
            score = 100.0  # 默认分数
            try:
                score_match = re.search(r'总分: (\d+\.?\d*)', row['业务操作类型'])
                if score_match:
                    score = float(score_match.group(1))
                elif "直接放行" in row['业务操作类型']:
                    score = 100.0  # 直接放行给高分
            except Exception as e:
                print(f"提取分数时出错: {str(e)}")
            
            ai_audit_list.append({
                "signature": row['短信签名'],
                "content": row['短信内容'],
                "business_type": row['产品类型'],
                "rule_score": score,
                "rule_reason": row['业务操作类型']
            })
        except Exception as e:
            print(f"准备AI审核数据时出错: {str(e)}")
    return ai_audit_list


# 4. 测试小批量审核
def test_batch_audit(df):
    """测试小批量审核"""    
    # 然后在下一个单元格中可以随机抽取部分放行的短信进行AI二次审核
    sample_size = 10  # 设置样本大小
    passed_df = df[df['总体操作类型'] == '放行']
    sample_df = passed_df
    sample_df = sample_df.sample(min(sample_size, len(passed_df)))
    print(f"已随机抽取 {len(sample_df)} 条放行规则审核的短信用于AI二次审核")
    print("============= 开始第二轮：AI审核 =============")
    # 准备AI审核数据
    ai_audit_list = prepare_ai_data(sample_df)

    # 批量审核
    auditor = AIAuditor()
    start_time = time.time()
    results = auditor.batch_audit(ai_audit_list)
    elapsed = time.time() - start_time
    
    # 统计结果
    pass_count = sum(1 for result in results if result['passed'])
    reject_count = len(results) - pass_count
    
    print(f"审核完成: 放行 {pass_count} 条, 失败 {reject_count} 条")
    print(f"总耗时: {elapsed:.2f}秒, 平均每条: {elapsed/len(results):.2f}秒")
    
    # 显示详细结果
    print("\n详细结果:")
    for i, result in enumerate(results):
        sms = result['sms']
        print(f"\n短信 {i+1}:")
        print(f"签名: {sms['signature']}")
        print(f"内容: {sms['content'][:50]}..." if len(sms['content']) > 50 else f"内容: {sms['content']}")
        print(f"业务类型: {sms['business_type']}")
        print(f"审核结果: {'放行' if result['passed'] else '失败'}")
        if not result['passed'] and 'reasons' in result['details']:
            print(f"失败原因: {', '.join(result['details']['reasons'])}")
    
    return results

# 运行测试
# test_single_audit()
test_batch_audit(df_with_results)

In [ ]:
# 3. 测试单条短信审核
import time
from business import validate_business
from typing import Tuple, Dict
import pandas as pd
import os
from datetime import datetime
import re
from collections import Counter
from ai_check import AIAuditor
import json

def test_single_audit():
    """测试单条短信审核"""
    # 测试一条短信
    test_sms = {
        "signature": " 饿了么 ",
        "content": "【饿了么】尊敬的超级吃货卡用户，我们将在2025-04-02放行支付宝扣除你对应下一周期的卡费，扣款次日即可领取4个红包，扣除的卡费及卡种类请见https://to.ele.me/WPT1ZT9 ",
        "business_type": "行业-物流",
        "account_type": "直客"
    }
    print("=============== 短信信息 =============")
    print(f"签名: {test_sms['signature']}")
    print(f"内容: {test_sms['content']}")
    print(f"业务类型: {test_sms['business_type']}")
    if test_sms.get("account_type"):
        print(f"账户类型: {test_sms['account_type']}")
    print("\n")
    print("=============== 第一轮：规则审核 =============")
    # 执行规则审核
    business_passed, business_reason = validate_business(
        test_sms["business_type"],
        test_sms["content"],
        test_sms["signature"],
        test_sms.get("account_type")
    )   

    
    print(f"规则审核结果: {'放行' if business_passed else '失败'}")
    print(f"规则审核原因: {business_reason}")

# 运行测试
test_single_audit()

In [4]:
#测试DeepSeek API连接
import requests
import time
from typing import Dict, Any

# API配置
API_URL = "https://api.deepseek.com/chat/completions"
API_KEY = "sk-fdb5269b9e0e43aca3cf7dea21d63322"  # 替换为实际API密钥
TIMEOUT = 10  # 请求超时时间(秒)

def test_api():
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {API_KEY}"}
    payload = {"model": "deepseek-chat", "messages": [{"role": "user", "content": "Hello"}]}
    
    try:
        print(f"测试API连接，超时: {TIMEOUT}秒...")
        start_time = time.time()
        response = requests.post(API_URL, headers=headers, json=payload, timeout=TIMEOUT)
        elapsed = time.time() - start_time
        
        print(f"API响应: 状态码={response.status_code}, 耗时={elapsed:.2f}秒")
        return response.status_code == 200, elapsed
    except Exception as e:
        print(f"API连接错误: {str(e)}")
        return False, 0

if __name__ == "__main__":
    success, time_used = test_api()
    print(f"API测试结果: {'成功' if success else '失败'}")

测试API连接，超时: 10秒...
API响应: 状态码=200, 耗时=4.07秒
API测试结果: 成功


In [1]:
import importlib
import ai_check 
importlib.reload(ai_check )

/Users/a92461/Documents/sohan/messagecheck/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'ai_check' from '/Users/a92461/Documents/sohan/messagecheck/ai_check.py'>

In [2]:
import time
import json
from ai_check import AIAuditor

def test_single_sms():
    """测试单条短信AI审核"""
    # 测试一条短信
    test_sms = {
        "signature": "广东商城",
        "content": "【广东商城】(温馨提示)尊敬的用户，您的电话号码卡已经办理成功，请插卡到手机使用，流量等优惠于48小时内到账，可关注微信公众号\"gdkf10000\"查询，感谢您的支持！",
        "business_type": "行业-通知"
    }
    
    print("=============== 短信信息 =============")
    print(f"签名: {test_sms['signature']}")
    print(f"内容: {test_sms['content']}")
    print(f"业务类型: {test_sms['business_type']}")
    print("\n")
   
    # 创建审核器
    auditor = AIAuditor()
    
    # 使用常规方法审核
    print("=============== AI审核 =============")
    start_time = time.time()
    passed, details = auditor.audit_sms(
        test_sms["signature"],
        test_sms["content"],
        test_sms["business_type"]
    )
    elapsed = time.time() - start_time
    
    print(f"AI审核结果: {'通过' if passed else '失败'}")
    print(f"AI审核详情: {json.dumps(details, ensure_ascii=False, indent=2)}")
    print(f"AI审核耗时: {elapsed:.2f}秒")
    
    print("\n=============== Token消耗 =============")
    standard_tokens = details.get("token_usage", {}).get("total_tokens", 0)
    
    print(f"Token消耗: {standard_tokens}")
    
    print("\n=============== 最终审核结果 =============")
    print(f"审核结果: {'通过' if passed else '失败'}")
    if not passed:
        print(f"失败原因: {', '.join(details.get('reasons', []))}")

if __name__ == "__main__":
    test_single_sms()


2025-04-27 09:26:18,933 - INFO - Use pytorch device_name: mps
2025-04-27 09:26:18,934 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-zh-v1.5


=============== 短信信息 =============
签名: 广东商城
内容: 【广东商城】(温馨提示)尊敬的用户，您的电话号码卡已经办理成功，请插卡到手机使用，流量等优惠于48小时内到账，可关注微信公众号"gdkf10000"查询，感谢您的支持！
业务类型: 行业-通知




2025-04-27 09:27:14,482 - INFO - 向量化和缓存功能已初始化


=============== AI审核 =============


2025-04-27 09:27:19,188 - INFO - API调用耗时: 4.71秒
2025-04-27 09:27:19,190 - INFO - 标准审核 Token使用: 输入=401, 输出=64, 总计=465


AI审核结果: 失败
AI审核详情: {
  "should_pass": false,
  "risk_areas": [
    "业务特殊规则",
    "通用审核标准"
  ],
  "reasons": [
    "短信内容中包含微信公众号关注信息，属于敏感内容",
    "短信内容中使用了中括号，违反通用审核标准"
  ],
  "token_usage": {
    "input_tokens": 401,
    "output_tokens": 64,
    "total_tokens": 465
  },
  "api_time": 4.70503306388855
}
AI审核耗时: 4.71秒

=============== Token消耗 =============
Token消耗: 465

=============== 最终审核结果 =============
审核结果: 失败
失败原因: 短信内容中包含微信公众号关注信息，属于敏感内容, 短信内容中使用了中括号，违反通用审核标准


In [7]:
import importlib
import ai_check 
importlib.reload(ai_check)

<module 'ai_check' from '/Users/a92461/Documents/sohan/messagecheck/ai_check.py'>

In [5]:
import time
import pandas as pd
from datetime import datetime
from ai_check import AIAuditor
import psutil

def test_Excel_batch(file_path="3月审核记录.xlsx", sample_size=1000, random_sample=True):
    """
    测试批量Excel文件处理
    
    Args:
        file_path: Excel文件路径
        sample_size: 要处理的样本数量，None表示全部
        random_sample: 是否使用随机抽样，默认为True
    """
    process = psutil.Process()
    
    # 记录初始内存使用
    mem_before = process.memory_info().rss / 1024 / 1024
    print(f"初始内存使用: {mem_before:.2f} MB")
    
    # 读取Excel文件
    try:
        # 先读取全部数据
        df = pd.read_excel(file_path)
        total_records = len(df)
        print(f"读取Excel文件成功，共{total_records}条记录")
        
        # 如果需要抽样且指定了样本大小
        if random_sample and sample_size and sample_size < total_records:
            df = df.sample(n=sample_size, random_state=42)  # random_state确保结果可重现
            print(f"已随机抽取{sample_size}条记录进行测试")
    except Exception as e:
        print(f"读取Excel文件失败: {str(e)}")
        return
    
    
    # 准备AI审核数据
    ai_audit_list = []
    for _, row in df.iterrows():
        ai_audit_list.append({
            "signature": row['短信签名'],
            "content": row['短信内容'],
            "business_type": row['产品类型'],
        })
    
    # 创建审核器（只创建一次，保留缓存）
    auditor = AIAuditor()
    

    print(f"\n=============== 开始第AI批量审核 =============")
    start_time = time.time()
    results = auditor.batch_audit(ai_audit_list)
    elapsed = time.time() - start_time
    
    # 基础统计结果
    pass_count = sum(1 for result in results if result['passed'])
    reject_count = len(results) - pass_count
    reject_rate = (reject_count / len(results)) * 100 if len(results) > 0 else 0
    
    # 统计缓存命中情况
    cache_hits = sum(1 for r in results if r.get("details", {}).get("cached", False))
    cache_rate = (cache_hits / len(results)) * 100 if len(results) > 0 else 0
    
    print(f"审核完成: 放行 {pass_count} 条, 失败 {reject_count} 条")
    print(f"拦截率: {reject_rate:.2f}%")
    print(f"缓存命中: {cache_hits}/{len(results)} ({cache_rate:.2f}%)")
    print(f"总耗时: {elapsed:.2f}秒, 平均每条: {elapsed/len(results):.2f}秒")
    
    # 记录当前内存使用
    mem_after = process.memory_info().rss / 1024 / 1024
    print(f"当前内存使用: {mem_after:.2f} MB (增加 {mem_after - mem_before:.2f} MB)")
    
    
    # 与操作类型比较的统计
    if '操作类型' in df.columns:
        # 创建结果对比
        df['AI审核结果'] = ['放行' if results[i]['passed'] else '失败' for i in range(len(results))]
        
        # 计算匹配率相关统计
        ai_pass_op_pass = sum(1 for i, result in enumerate(results) 
                           if result['passed'] and df.iloc[i]['操作类型'] == '放行')
        ai_reject_op_pass = sum(1 for i, result in enumerate(results) 
                             if not result['passed'] and df.iloc[i]['操作类型'] == '放行')
        ai_pass_op_reject = sum(1 for i, result in enumerate(results) 
                             if result['passed'] and df.iloc[i]['操作类型'] == '失败')
        ai_reject_op_reject = sum(1 for i, result in enumerate(results) 
                               if not result['passed'] and df.iloc[i]['操作类型'] == '失败')
        
        total_op_pass = sum(1 for i in range(len(results)) if df.iloc[i]['操作类型'] == '放行')
        total_op_reject = sum(1 for i in range(len(results)) if df.iloc[i]['操作类型'] == '失败')
        
        match_count = ai_pass_op_pass + ai_reject_op_reject
        match_rate = (match_count / len(results)) * 100 if len(results) > 0 else 0
        
        print("\n=============== AI与人工操作对比 =============")
        print(f"总记录数: {len(results)}")
        print(f"AI放行 & 人工放行: {ai_pass_op_pass} 条")
        print(f"AI失败 & 人工放行: {ai_reject_op_pass} 条")
        print(f"AI放行 & 人工失败: {ai_pass_op_reject} 条")
        print(f"AI失败 & 人工失败: {ai_reject_op_reject} 条")
        print(f"匹配率: {match_rate:.2f}%")
        
        # 细分统计
        if total_op_pass > 0:
            correct_pass_rate = (ai_pass_op_pass / total_op_pass) * 100
            print(f"人工放行中AI正确判断率: {correct_pass_rate:.2f}%")
        
        if total_op_reject > 0:
            correct_reject_rate = (ai_reject_op_reject / total_op_reject) * 100
            print(f"人工失败中AI正确判断率: {correct_reject_rate:.2f}%")

        if total_op_reject > 0:
            ai_wrong_rate = (ai_pass_op_reject / len(results)) * 100
            print(f"AI漏杀率: {ai_wrong_rate:.2f}%")
        
    # 添加结果列到DataFrame并保存
    df['AI审核结果'] = ['放行' if results[i]['passed'] else '失败' for i in range(len(results))]
    
    # 添加失败原因
    reasons = []
    for result in results:
        if not result['passed'] and 'reasons' in result['details']:
            reasons.append(', '.join(result['details']['reasons']))
        else:
            reasons.append('')
    df['失败原因'] = reasons

    # 添加匹配情况
    if '操作类型' in df.columns:
        df['AI与人工是否匹配'] = [1 if ((result['passed'] and row['操作类型'] == '放行') or 
                           (not result['passed'] and row['操作类型'] == '失败')) else 0
                           for result, (_, row) in zip(results, df.iterrows())]
    # 保存结果
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"AI审核结果_{timestamp}.xlsx"
    df.to_excel(output_file, index=False)
    print(f"\n审核结果已保存至: {output_file}")

if __name__ == "__main__":
   
    test_Excel_batch()
 

初始内存使用: 486.73 MB


2025-04-27 09:36:57,700 - INFO - 正在加载模型: BAAI/bge-small-zh-v1.5
2025-04-27 09:36:57,701 - INFO - Use pytorch device_name: mps
2025-04-27 09:36:57,702 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-zh-v1.5


读取Excel文件成功，共8133条记录
已随机抽取1000条记录进行测试


2025-04-27 09:37:03,062 - INFO - 成功加载模型: BAAI/bge-small-zh-v1.5, 向量维度: 512
2025-04-27 09:37:03,063 - INFO - 向量化和缓存功能已初始化
2025-04-27 09:37:03,063 - INFO - 开始批量审核 1000 条短信
2025-04-27 09:37:03,063 - INFO - 进度: 1/1000 [0.0% 缓存命中] [平均 0.00秒/条] [预计剩余 0秒]



=============== 开始第AI批量审核 =============


2025-04-27 09:37:11,472 - INFO - API调用耗时: 8.41秒
2025-04-27 09:37:11,474 - INFO - 标准审核 Token使用: 输入=381, 输出=108, 总计=489
2025-04-27 09:37:16,939 - INFO - API调用耗时: 5.39秒
2025-04-27 09:37:16,941 - INFO - 标准审核 Token使用: 输入=377, 输出=45, 总计=422
2025-04-27 09:37:24,686 - INFO - API调用耗时: 7.69秒
2025-04-27 09:37:24,688 - INFO - 标准审核 Token使用: 输入=410, 输出=88, 总计=498
2025-04-27 09:37:31,131 - INFO - API调用耗时: 6.38秒
2025-04-27 09:37:31,133 - INFO - 标准审核 Token使用: 输入=376, 输出=51, 总计=427
2025-04-27 09:37:31,196 - INFO - 从缓存获取结果，相似度: 0.6618
2025-04-27 09:37:40,146 - INFO - API调用耗时: 8.91秒
2025-04-27 09:37:40,148 - INFO - 标准审核 Token使用: 输入=396, 输出=127, 总计=523
2025-04-27 09:37:40,198 - INFO - 从缓存获取结果，相似度: 0.8157
2025-04-27 09:37:40,238 - INFO - 从缓存获取结果，相似度: 0.7800
2025-04-27 09:37:40,245 - INFO - 从缓存获取结果，相似度: 0.7543
2025-04-27 09:37:50,080 - INFO - API调用耗时: 9.58秒
2025-04-27 09:37:50,081 - INFO - 标准审核 Token使用: 输入=492, 输出=100, 总计=592
2025-04-27 09:37:50,136 - INFO - 进度: 11/1000 [36.4% 缓存命中] [平均 4.28秒/条] [预计剩余 4232秒]

审核完成: 放行 671 条, 失败 329 条
拦截率: 32.90%
缓存命中: 972/1000 (97.20%)
总耗时: 2212.73秒, 平均每条: 2.21秒
当前内存使用: 1155.80 MB (增加 669.06 MB)

=============== AI与人工操作对比 =============
总记录数: 1000
AI放行 & 人工放行: 567 条
AI失败 & 人工放行: 297 条
AI放行 & 人工失败: 104 条
AI失败 & 人工失败: 32 条
匹配率: 59.90%
人工放行中AI正确判断率: 65.62%
人工失败中AI正确判断率: 23.53%
AI漏杀率: 10.40%

审核结果已保存至: AI审核结果_20250427_101355.xlsx
